In [1]:
#import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import datetime as dt
import seaborn as sns
from cycler import cycler
# from config import gkey
# from geopy import geocoders
# gmaps.configure(api_key=gkey)

In [2]:
file="SaferParks.csv"
pd.set_option('display.max_columns', None)
dfs = pd.read_csv(file,encoding = 'cp1252')
dfs.head()


,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,tradename_or_generic,manufacturer,num_injured,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes
0,1005813,6/12/2010,OH,Cleveland,F,Ohio Dept. of Agriculture,Sports or recreation facility,recreation,inflatable,Inflatable slide,inflatable slide,Scherba Industries / Inflatable Images,9.0,NaN,NaN,Inflatable slide tipped over while 7-9 patrons...,The man who was crushed by the device died 9 d...,https://ridesdatabase.org/accident_reports/OH-...,"Device tipped over, blew away, or collapsed",NaN,1.0,NaN,http://www.cleveland.com/metro/index.ssf/2012/...
1,1004032,6/12/2010,OH,Cleveland,P,United States Consumer Product Safety Commission,Sports or recreation facility,recreation,inflatable,Inflatable slide,inflatable slide,Scherba Industries / Inflatable Images,8.0,54.0,M,Two adult males were standing next to an infla...,1 - death from pulmonary embolism caused by bl...,https://ridesdatabase.org/accident_reports/CPS...,"Device tipped over, blew away, or collapsed",NaN,1.0,NaN,See also OH state record.
2,1007658,7/10/2010,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,water ride,Boat ride,boat ride,Arrow Dynamics,1.0,37.0,F,Patron stated she slipped and fell while stepp...,"Hip and low back pain, torn meniscus in knee",NaN,Load/Unload: scrape or stumble,NaN,NaN,NaN,NaN
3,1007098,7/10/2010,CA,Carlsbad,F,California Division of Occupational Safety and...,Water park,water park,float attraction,Lazy river,lazy river,California Pools,1.0,37.0,F,Patron claims strained back after her raft tip...,Strained back,NaN,"Fall: patron fell off inner tube, mat or board",NaN,NaN,NaN,NaN
4,1000094,7/29/2010,CO,Littleton,F,Colorado Division of Oil and Public Safety,Family entertainment center,recreation,challenge activity,Climbing wall,climbing wall,Spectrum Sports,1.0,NaN,M,"Two patrons climbed to the top of the wall, co...",Patron experienced a free fall that resulted i...,https://ridesdatabase.org/accident_reports/CO-...,Fall: patron fell from device (participatory),1.0,NaN,NaN,NaN


In [3]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8351 entries, 0 to 8350
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   acc_id                8351 non-null   int64  
 1   acc_date              8351 non-null   object 
 2   acc_state             8351 non-null   object 
 3   acc_city              8233 non-null   object 
 4   fix_port              8351 non-null   object 
 5   source                8351 non-null   object 
 6   bus_type              8351 non-null   object 
 7   industry_sector       8351 non-null   object 
 8   device_category       8351 non-null   object 
 9   device_type           8351 non-null   object 
 10  tradename_or_generic  8351 non-null   object 
 11  manufacturer          5041 non-null   object 
 12  num_injured           8349 non-null   float64
 13  age_youngest          7667 non-null   float64
 14  gender                7623 non-null   object 
 15  acc_desc             

In [4]:
dff=dfs.sample(n=1000)
dff

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,tradename_or_generic,manufacturer,num_injured,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes
6202,1001762,8/9/2015,OH,Sandusky,F,Ohio Dept. of Agriculture,Amusement park,amusement ride,coaster,Coaster - steel,steel sitdown coaster,Bolliger & Mabillard,1.0,28.0,F,Patron suffered chest pain and rapid heartbeat...,"Chest pain, rapid heartbeat. Diagnosis was red...",NaN,Restraint too tight,NaN,NaN,NaN,NaN
649,1006858,8/10/2011,CA,Santa Cruz,F,California Division of Occupational Safety and...,Amusement park,amusement ride,vertical drop,Drop tower,Double Shot,S&S Worldwide,1.0,58.0,F,"At end of the ride, as patron was getting off,...",Seizure or syncope,NaN,Illness: Seizure or LOC,NaN,NaN,NaN,NaN
6437,1009099,9/13/2015,CA,Buena Park,F,California Division of Occupational Safety and...,Amusement park,amusement ride,coaster,Coaster - steel,spinning coaster,Mack Rides,1.0,25.0,F,Patron was walking through a turnstile when ei...,Leg injury,NaN,Entrapment or pinch-point,NaN,NaN,NaN,NaN
982,1007379,2/14/2012,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Train/tram,monorail,In-house,1.0,15.0,F,Patron stated she stepped wrong as she was exi...,Injury to heel,NaN,Load/Unload: scrape or stumble,NaN,NaN,NaN,NaN
5136,1001760,3/14/2015,OH,Cincinnati,F,Ohio Dept. of Agriculture,Pool waterslide,water park,water slide,Body slide,body slide,WhiteWater West Industries Ltd.,1.0,NaN,M,Adult patron had difficulty breathing after ex...,Breathing difficulties,NaN,Illness or neurological symptoms,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,1008905,6/8/2015,CA,San Diego,F,California Division of Occupational Safety and...,Amusement park,amusement ride,water ride,Rafting ride,rafting ride,Intamin AG,1.0,39.0,F,While going through the tunnel towards the end...,Seizure,NaN,Illness: Seizure or LOC,NaN,NaN,NaN,NaN
7112,1002398,5/14/2016,PA,Leetsdale,F,Pennsylvania Dept. of Agriculture,Trampoline park,recreation,trampoline,Trampoline court,TC - foam pit,NaN,1.0,41.0,M,Patron injured his right knee on his last jump...,Knee injury; unable to walk on it,NaN,"Injury to leg, nonspecific cause",NaN,NaN,NaN,NaN
2516,1008160,5/19/2013,CA,Valencia,F,California Division of Occupational Safety and...,Amusement park,amusement ride,water ride,Flume ride,Hydro Flume,Arrow Dynamics,1.0,30.0,F,Patron stated she was exiting the boat in the ...,Injury to shoulder and head,NaN,Load/Unload: injured when vehicle moved,NaN,NaN,NaN,NaN
2845,1000722,7/6/2013,FL,SILVER SPRINGS,F,Florida Dept. of Agriculture,Water park,water park,water slide,Bowl slide,bowl-type slide,NaN,1.0,26.0,F,26 year old female flipped her tube while in t...,Laceration to eye,NaN,"Fall: patron fell off inner tube, mat or board",NaN,NaN,NaN,NaN


In [5]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 6202 to 4218
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   acc_id                1000 non-null   int64  
 1   acc_date              1000 non-null   object 
 2   acc_state             1000 non-null   object 
 3   acc_city              987 non-null    object 
 4   fix_port              1000 non-null   object 
 5   source                1000 non-null   object 
 6   bus_type              1000 non-null   object 
 7   industry_sector       1000 non-null   object 
 8   device_category       1000 non-null   object 
 9   device_type           1000 non-null   object 
 10  tradename_or_generic  1000 non-null   object 
 11  manufacturer          590 non-null    object 
 12  num_injured           1000 non-null   float64
 13  age_youngest          915 non-null    float64
 14  gender                915 non-null    object 
 15  acc_desc          

In [6]:
dff.dropna(axis=0, subset="acc_city", inplace=True)

In [7]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 987 entries, 6202 to 4218
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   acc_id                987 non-null    int64  
 1   acc_date              987 non-null    object 
 2   acc_state             987 non-null    object 
 3   acc_city              987 non-null    object 
 4   fix_port              987 non-null    object 
 5   source                987 non-null    object 
 6   bus_type              987 non-null    object 
 7   industry_sector       987 non-null    object 
 8   device_category       987 non-null    object 
 9   device_type           987 non-null    object 
 10  tradename_or_generic  987 non-null    object 
 11  manufacturer          586 non-null    object 
 12  num_injured           987 non-null    float64
 13  age_youngest          902 non-null    float64
 14  gender                905 non-null    object 
 15  acc_desc           

In [8]:
#Make an address
dff["Address"] =dff.acc_city + ", " + dff.acc_state
dff.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,tradename_or_generic,manufacturer,num_injured,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes,Address
6202,1001762,8/9/2015,OH,Sandusky,F,Ohio Dept. of Agriculture,Amusement park,amusement ride,coaster,Coaster - steel,steel sitdown coaster,Bolliger & Mabillard,1.0,28.0,F,Patron suffered chest pain and rapid heartbeat...,"Chest pain, rapid heartbeat. Diagnosis was red...",NaN,Restraint too tight,NaN,NaN,NaN,NaN,"Sandusky, OH"
649,1006858,8/10/2011,CA,Santa Cruz,F,California Division of Occupational Safety and...,Amusement park,amusement ride,vertical drop,Drop tower,Double Shot,S&S Worldwide,1.0,58.0,F,"At end of the ride, as patron was getting off,...",Seizure or syncope,NaN,Illness: Seizure or LOC,NaN,NaN,NaN,NaN,"Santa Cruz, CA"
6437,1009099,9/13/2015,CA,Buena Park,F,California Division of Occupational Safety and...,Amusement park,amusement ride,coaster,Coaster - steel,spinning coaster,Mack Rides,1.0,25.0,F,Patron was walking through a turnstile when ei...,Leg injury,NaN,Entrapment or pinch-point,NaN,NaN,NaN,NaN,"Buena Park, CA"
982,1007379,2/14/2012,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Train/tram,monorail,In-house,1.0,15.0,F,Patron stated she stepped wrong as she was exi...,Injury to heel,NaN,Load/Unload: scrape or stumble,NaN,NaN,NaN,NaN,"Anaheim, CA"
5136,1001760,3/14/2015,OH,Cincinnati,F,Ohio Dept. of Agriculture,Pool waterslide,water park,water slide,Body slide,body slide,WhiteWater West Industries Ltd.,1.0,NaN,M,Adult patron had difficulty breathing after ex...,Breathing difficulties,NaN,Illness or neurological symptoms,NaN,NaN,NaN,NaN,"Cincinnati, OH"


In [12]:
#Get location data from opwnweathermap.org
url = f"http://api.openweathermap.org/geo/1.0/direct?q={address}&appid={API_Key}"
response = requests.get(url)
data = response.json()
if (len(data) > 0):
    location = data[0]
    print(location)

NameError: name 'address' is not defined

In [13]:
# parse data
lat = location[“lat”]
lon = location[“lon”]
city = location[“name”]
state = location.get(“state”) # might not exist
country = location[“country”]
row = {“latitude”: lat,
      “longitude”: lon,
      “city”: city,
      “state”: state,
      “country”: country}
row

SyntaxError: invalid character '“' (U+201C) (782073902.py, line 2)

In [14]:
#Loop to get location info for all addresses
rows = []
for address in dff.address:
    print(address)
    # create URL
    url = f”http://api.openweathermap.org/geo/1.0/direct?q={address}&appid={API_KEY}”
    response = requests.get(url)
    # get data
    data = response.json()
    if (len(data) > 0):
        location = data[0]
        # parse data
        lat = location[“lat”]
        lon = location[“lon”]
        city = location[“name”]
        state = location.get(“state”)
        # make row
        row = {“latitude”: lat,
              “longitude”: lon,
              “clean_city”: city,
              “clean_state”: state}
    else:
        row = {“latitude”: np.nan,
              “longitude”: np.nan,
              “clean_city”: None,
              “clean_state”: None }
    rows.append(row)
    time.sleep(1)

SyntaxError: invalid character '”' (U+201D) (4161879359.py, line 6)